In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.9 MB/s eta 

In [2]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

import os
from getpass import getpass


In [3]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter Mistral API Key: ")


Enter Mistral API Key: ··········


In [4]:
!pip install pypdf


In [5]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/FUNDAMENTALS OF MANAGEMENT.pdf")   # upload your PDF to colab
docs = loader.load()
len(docs)


345

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(docs)
len(chunks)


1721

In [7]:
embeddings = MistralAIEmbeddings(model="mistral-embed")

vector_db = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k":3})


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [8]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)


/tmp/ipython-input-3186262939.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [9]:
template = """
You are a Memory-Augmented RAG assistant.

Conversation History:
{chat_history}

Relevant Retrieved Context:
{context}

User Query:
{question}

Provide a context-aware answer. ALWAYS use history if relevant.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template=template
)


In [10]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)


In [ ]:
while True:
    q = input("You: ")
    if q.lower() in ["exit", "quit"]:
        break
    resp = rag_chain.invoke({"question": q})
    print("Bot:", resp["answer"])


You: What does pdf says
Bot: Based on the **retrieved context** and **conversation history**, here’s what the PDF appears to describe:

1. **Immediate vs. Long-Term Guidance (Context 316)**:
   The text suggests that a leader (likely Murthy or another authority figure) does **not** provide detailed future plans but instead focuses on **immediate next steps** for the team. He also offers **personal praise** (e.g., to Mehta in Context 306).

2. **Conflict Between Singh and Murthy (Context 306)**:
   - **Singh** presented an analysis (possibly criticizing "B" shift or Mehta’s performance).
   - **Murthy** **disagreed strongly**, defended "B" shift, and **praised Mehta** for high output.
   - Murthy **advised Singh to focus on "A" shift** and leave Mehta alone.
   - Singh realized arguing further would be futile and instead **decided to experiment with a new idea** (unnamed in the snippet).

### Key Takeaways:
- The PDF discusses **leadership communication styles** (direct vs. strategic).
